# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [18]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [19]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [20]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [21]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling With Apache Cassandra To Achieve Query Requirements

## With the previous pre-processing step, the CSV file titled <font color=red>event_datafile_new.csv</font> located within the root directory is ready to work on. The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Setup Apache Cassandra Cluster and Keyspace for the project

#### Creating a Cluster

In [22]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [23]:
# TO-DO: Create a Keyspace 
try:
    session.execute('''
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    ''')
except Exception as e:
    print(e)

#### Set Keyspace

In [24]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except:
    print(e)

### Now we need to create tables to run the following queries.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


### Solution to Question1:
<font color=green>Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4</font>

* The question1 expects artist name, song title and song's length that's based on sessionid and itemInSession.
* With a NoSQL database, we need to consider the query in order to design and create the table required.
* With the consideration, the query to the question1 will be a SELECT statement as below:

`SELECT artist, song, length FROM music_app_history WHERE sessionId = '338' and itemInSession = '4' ;`

With the above SELECT query, we can design/model the table based on the following consideration:
- Column names: as the query is after artist name, song title and song's length, and based on sessionid and itemInSession, we will need them all as the names of the columns;
- Primary key: the PRIMARY key for the table should uniquely identify each row in the table. For this query we need results based on sessionId and itemInSession; so we neeed these both as the primary key

In [25]:
## Create a table based on requirement of query1, including sessionId, itemInSession, artist, song, length as columns and sessionId, itemInSession as primary key
query = "CREATE TABLE IF NOT EXISTS music_app_history"
query = query + "(sessionId text, itemInSession text, artist text, song text, length text, PRIMARY KEY(sessionId, itemInSession));"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [26]:
# Read through each line of the CSV file and insert the relevant data into the music_app_history table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Column element to be assigned for each column in the INSERT statement.
        session.execute(query, (line[8], line[3], line[0], line[9], line[5]))

In [27]:
## Run the SELECT statement to obtain that the info we are after
query = "SELECT artist, song, length FROM music_app_history WHERE sessionId = '338' and itemInSession = '4';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### Solution to Question2:
<font color=green>Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182</font>

* The question2 expects name of artist, song (sorted by itemInSession) and user (first and last name) which are based on userid and sessionid.
* With a NoSQL database, we need to consider the query in order to design and create the table required.
* With the consideration, the query to the question2 will be a SELECT statement as below:

`SELECT artist, song, firstname, lastname FROM song_playlist_session WHERE userid = '10' and sessionid = '182';`

With the above SELECT query, we can design/model the table based on the following consideration:
- Column names: as the query is after artist, song, firstname, lastname, and based on userid and sessionid, we will need them all as the names of the columns;
- Primary key: the query is based on userid, sessionId and sorted by itemInSession, so we will need to use them as Partition key and Clustering column. We should use both userId and sessioId as partition keys so session from the same user are stored together - when the database is large, this design will ensure a better performance comparing to storing userid and sessionId in different nodes. We use itemInSession as clustering column so the result will be sorted by itemInSession.

In [28]:
## Create a table based on requirement of query2, including userid, sessionid, artist, song, firstName and lastName as columns; userid, sessionid as partition keys and itemInSession as clustering column
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(userid text, sessionid text, iteminsession text, artist text, song text, firstname text, lastname text,  PRIMARY KEY((userid, sessionid),itemInSession));"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [29]:
# Read through each line of the CSV file and insert the relevant data into the song_playlist_session table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## column element to be assigned for each column in the INSERT statement.
        session.execute(query, (line[10], line[8], line[3], line[0], line[9], line[1], line[4]))
        

In [30]:
## Run the SELECT statement to obtain that the info we are after
query = "SELECT artist, song, firstname, lastname FROM song_playlist_session WHERE userid = '10' and sessionid = '182';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Solution to Question3:
<font color=green>Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'</font>

* The question3 expects user (first and last name) which are based on the name of the song.
* With a NoSQL database, we need to consider the query in order to design and create the table required.
* With the consideration, the query to the question3 will be a SELECT statement as below:

`SELECT firstname, lastname FROM user_song_name WHERE song='All Hands Against His Own';`

With the above SELECT query, we can design/model the table based on the following consideration:
- Column names: as the query is after firstname, lastname, and based on song, we will need them all as the names of the columns;
- Primary key: the query is based on song, however, with only song key it cannot uniquely identify each records, so we will need to use another key together as primary key. The column, user id could uniquely identify each user. So the columns user_id and song_title are sufficient to identify each record uniquely, leading us to PRIMARY KEY (song_title, user_id).

In [31]:
## Create a table 'user_song_name' based on requirement of query3, including song, firstName and lastName as columns; song , userid as primary keys
query = "CREATE TABLE IF NOT EXISTS user_song_name"
query = query + "(song text, userid text, firstname text, lastname text, PRIMARY KEY(song , userid));"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [32]:
# Read through each line of the CSV file and insert the relevant data into the song_playlist_session table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_song_name (song, userid, firstname, lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], line[10], line[1], line[4]))    
        

In [33]:
## Run the SELECT statement to obtain that the info we are after
query = "SELECT firstname, lastname FROM user_song_name WHERE song='All Hands Against His Own' ;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print( row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [34]:
## Drop the table before closing out the sessions
## OR Drop table to reset if needed
query = '''
drop table music_app_history; 
'''
try:
    session.execute(query)
except Exception as e:
    print(e)

query = '''
drop table song_playlist_session; 
'''
try:
    session.execute(query)
except Exception as e:
    print(e)

query = '''
drop table user_song_name; 
'''
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [35]:
session.shutdown()
cluster.shutdown()